# 1. Parameters

In [1]:
from pathlib import Path

# Defaults
cases_dir = 'cases/full'
metadata_file = 'metadata.tsv'
iterations = 3
mincov = 10
ncores = 32

reference_files = {
    '0810PADBR-1': Path('data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz'),
    '1405WAEXK-1': Path('data/reference/ecoli/GCF_000703365.1_Ec2011C-3609_genomic.gbk.gz'),
    '1408MLGX6-3WGS': Path('data/reference/listeria/GCF_001047715.2_ASM104771v2_genomic.gbk.gz'),
    '1203NYJAP-1 - Tuna Scrape Outbreak': Path('data/reference/salmonella/GCF_000439415.1_ASM43941v1_genomic.gbk.gz'),
}
    
build_tree = True
sample_batch_size=10

In [2]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)
index_path = cases_dir_path / 'index'

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

case_name = str(cases_dir_path.name)

benchmark_path = cases_dir_path / 'index-info.tsv'

output_trees = {x: cases_dir_path / f'{x}_tree.tre'.replace(' ', '_') for x in reference_files}
output_trees

{'0810PADBR-1': PosixPath('cases/full/0810PADBR-1_tree.tre'),
 '1405WAEXK-1': PosixPath('cases/full/1405WAEXK-1_tree.tre'),
 '1408MLGX6-3WGS': PosixPath('cases/full/1408MLGX6-3WGS_tree.tre'),
 '1203NYJAP-1 - Tuna Scrape Outbreak': PosixPath('cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_tree.tre')}

# 2. Create subset inputs

In [3]:
import pandas as pd
from pathlib import Path
from typing import Dict

metadata_df = pd.read_csv(metadata_file, sep='\t')

def write_subset_input(metadata_df: pd.DataFrame, dataset_name: str) -> Path:
    all_input_total = len(metadata_df)
    cases_input = cases_dir_path / f'{dataset_name}_input-files-case.tsv'.replace(' ', '_')

    input_df = metadata_df.copy().loc[metadata_df['dataSetName'] == dataset_name]
    input_df['Sample'] = input_df['strain']
    input_df['Assemblies'] = pd.NA
    input_df['Reads1'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_1.fastq.gz')).absolute()))
    input_df['Reads2'] = input_df['Sample'].apply(lambda x: str((Path('data') / 'fastq' / (x + '_2.fastq.gz')).absolute()))
    input_df = input_df[['Sample', 'Assemblies', 'Reads1', 'Reads2']]

    input_df.to_csv(cases_input, sep='\t', index=False)

    subset_input_total = len(input_df)

    print(f'Wrote dataset={dataset_name} consisting of {subset_input_total}/{all_input_total} samples to {cases_input}')
    
    return cases_input

cases_inputs = {x: write_subset_input(metadata_df, dataset_name=x) for x in reference_files}
cases_inputs

Wrote dataset=0810PADBR-1 consisting of 22/85 samples to cases/full/0810PADBR-1_input-files-case.tsv
Wrote dataset=1405WAEXK-1 consisting of 9/85 samples to cases/full/1405WAEXK-1_input-files-case.tsv
Wrote dataset=1408MLGX6-3WGS consisting of 31/85 samples to cases/full/1408MLGX6-3WGS_input-files-case.tsv
Wrote dataset=1203NYJAP-1 - Tuna Scrape Outbreak consisting of 23/85 samples to cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_input-files-case.tsv


{'0810PADBR-1': PosixPath('cases/full/0810PADBR-1_input-files-case.tsv'),
 '1405WAEXK-1': PosixPath('cases/full/1405WAEXK-1_input-files-case.tsv'),
 '1408MLGX6-3WGS': PosixPath('cases/full/1408MLGX6-3WGS_input-files-case.tsv'),
 '1203NYJAP-1 - Tuna Scrape Outbreak': PosixPath('cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_input-files-case.tsv')}

# 2. Index genomes

In [4]:
!gdi --version

gdi, version 0.5.0


## 2.1. Index reads

In [5]:
import time

start = time.time()
benchmarker = gdi_benchmark.IndexBenchmarkerMultiple(index_path=index_path, input_files_files=cases_inputs,
                                             reference_files=reference_files, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores,
                                             sample_batch_size=sample_batch_size)
benchmark_df = benchmarker.benchmark(iterations=iterations)
end = time.time()
print(f'***Finished benchmarking, took {(end - start)/60:0.2f} minutes***')

Creating new index: [gdi init cases/full/index]


Creating a new index took 3.32 seconds



Iteration 1 of index/analysis of 22 samples for reference=data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz with 32 cores
Removing any extra snakemake directories: []
Analysis running: [gdi --project-dir cases/full/index --ncores 32 analysis --use-conda --no-load-data --reference-file data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/full/0810PADBR-1_input-files-case.tsv]


Analysis took 16.15 minutes
Index running: [gdi --project-dir cases/full/index --ncores 32 load vcf-kmer --sample-batch-size 10 --reference-file data/reference/campylobacter/GCA_001879185.2_ASM187918v2_genomic.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/reads/snakemake-assemblies.1644355456.9301004/gdi-input.fofn]


Indexing took 0.88 minutes
Building tree: [gdi --project-dir cases/full/index --ncores 32 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' GCA_001879185.2_ASM187918v2_genomic]


...

Indexing took 4.93 minutes
Building tree: [gdi --project-dir cases/full/index --ncores 32 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' GCF_000439415.1_ASM43941v1_genomic]


Building tree took 1.76 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f3af2397c40> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


***Finished benchmarking, took 206.18 minutes***


In [6]:
benchmark_df

,Name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),Analysis memory (max/process),Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,0810PADBR-1,1,22,45117,1102,32,1634890,969.08,3.163837e+10,3.428127e+09,1.201150e+10,52.91,1.235374e+10,7.402291e+08,46665728.0,105.17,5.443912e+08,275349504.0,1127.16,3.163837e+10
0,1405WAEXK-1,1,9,45117,1102,32,5412686,735.64,9.107980e+09,1.683624e+09,6.563504e+09,216.23,9.673462e+09,3.800670e+09,134737920.0,96.33,8.176353e+08,459063296.0,1048.20,9.673462e+09
0,1408MLGX6-3WGS,1,31,45117,1102,32,2939733,743.46,2.859383e+10,1.928933e+09,1.081554e+10,116.57,1.680535e+10,9.866117e+08,174526464.0,126.51,9.937183e+08,660197376.0,986.54,2.859383e+10
0,1203NYJAP-1 - Tuna Scrape Outbreak,1,23,45117,1102,32,4808805,542.71,2.064738e+10,1.370657e+09,9.024676e+09,294.53,4.627037e+10,3.002397e+09,252043264.0,122.26,1.333158e+09,869711872.0,959.50,4.627037e+10
0,0810PADBR-1,2,22,45117,1102,32,1634890,918.73,3.078411e+10,3.428184e+09,1.201149e+10,51.36,1.377599e+10,7.449108e+08,46403584.0,106.84,5.440922e+08,275890176.0,1076.93,3.078411e+10
0,1405WAEXK-1,2,9,45117,1102,32,5412686,718.66,8.670171e+09,1.683505e+09,6.563508e+09,212.72,8.914026e+09,3.803709e+09,134504448.0,116.89,8.169308e+08,459284480.0,1048.27,8.914026e+09
0,1408MLGX6-3WGS,2,31,45117,1102,32,2939733,749.41,2.807024e+10,1.928933e+09,1.081550e+10,118.72,1.680661e+10,1.037353e+09,173465600.0,125.41,9.928786e+08,660254720.0,993.54,2.807024e+10
0,1203NYJAP-1 - Tuna Scrape Outbreak,2,23,45117,1102,32,4808805,562.04,1.919409e+10,1.370464e+09,9.024688e+09,296.85,3.771182e+10,2.726552e+09,249270272.0,129.44,1.334055e+09,869912576.0,988.33,3.771182e+10
0,0810PADBR-1,3,22,45117,1102,32,1634890,919.80,3.076841e+10,3.428151e+09,1.201148e+10,55.00,1.507837e+10,7.402127e+08,46637056.0,80.24,5.438095e+08,275492864.0,1055.04,3.076841e+10
0,1405WAEXK-1,3,9,45117,1102,32,5412686,731.42,8.699195e+09,1.683472e+09,6.563488e+09,211.65,9.736053e+09,3.804434e+09,134230016.0,111.76,8.185774e+08,459481088.0,1054.83,9.736053e+09


In [7]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [8]:
if build_tree:
    # Using variables with shell commands in Jupyter isn't working for me so I have to avoid using them
    !gdi --project-dir cases/full/index export tree GCA_001879185.2_ASM187918v2_genomic > cases/full/0810PADBR-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000703365.1_Ec2011C-3609_genomic > cases/full/1405WAEXK-1_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_001047715.2_ASM104771v2_genomic > cases/full/1408MLGX6-3WGS_tree.tre
    !gdi --project-dir cases/full/index export tree GCF_000439415.1_ASM43941v1_genomic > cases/full/1203NYJAP-1_-_Tuna_Scrape_Outbreak_tree.tre
else:
    print(f'build_tree={build_tree} so no trees to export')